# GHCND Stations Data Prep
#### Description:
This notebook is used to prep the data contained in ghcnd-stations.txt. Use noaa-daily-retrieve-files-ftp.ipynb to download this file among other metadta files used in the NOAA Daily Weather Project. 

System of Origin Data Source: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

#### Data Prep Operations:
- define english column names
- create CountryCode column from first 2 characters of StationID
- merge with ghcnd-countries.txt to get CountryName and ghcnd-states.txt to get StateName
- save to .csv file (tab separated)

#### Created by:
Nate Muth <br>
nmuth87@gmail.com

#### Created on:
7/29/2018

#### Changelog:
7/29/2018 - Initial Create Date<br>
8/5/2018 - Added metadata tables to give aliases for countries and states

In [2]:
import pandas as pd

In [3]:
# read stations.txt into DataFrame
stationsDF = pd.read_fwf('ghcnd-stations.txt', header=None, delimiter=' '
                         , widths=[12,9,10,7,3,31,4,4,6]
                         , names=['StationID', 'Latitude', 'Longitude', 'Elevation',
                                 'State', 'StationName', 'GSN_Flag', 'HCN_CRN_Flag', 'WMO_ID']
                         , dtypes={'WMO_ID':object}
                         )

# set StationID as the index
stationsDF.set_index('StationID', inplace=True)

In [4]:
stationsDF.head()

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID
StationID,,,,,,,,
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [5]:
stationsDF[stationsDF['StationName'].str.contains('EPPLEY')]

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID
StationID,,,,,,,,
USW00014942,41.3103,-95.8992,299.3,NE,OMAHA EPPLEY AIRFIELD,NaN,NaN,72550.0


In [6]:
def countryCode(row):
    return row['temp_StationID'][:2]

stationsDF['temp_StationID'] = stationsDF.index.values
stationsDF['CountryCode']= stationsDF.apply(countryCode, axis=1)
stationsDF.drop(columns=['temp_StationID'],inplace=True)

In [7]:
stationsDF[stationsDF['StationName'].str.contains('EPPLEY')]

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID,CountryCode
StationID,,,,,,,,,
USW00014942,41.3103,-95.8992,299.3,NE,OMAHA EPPLEY AIRFIELD,NaN,NaN,72550.0,US


### 8/5/2018 | Added metadata tables to give aliases for countries and states
Joining to ghcnd-countries.txt on the CountryCode column<br>
Joining to ghcnd-states.txt on the State column

In [10]:
countriesDF = pd.read_fwf('ghcnd-countries.txt',header=None,delimiter=' ',names=['CountryCode','CountryName'])
countriesDF.head()

,CountryCode,CountryName
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan


In [13]:
stations_DF2 = pd.merge(stationsDF,countriesDF,on='CountryCode',how='left')
stations_DF2[stations_DF2['StationName'].str.contains('EPPLEY')]

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID,CountryCode,CountryName
104868,41.3103,-95.8992,299.3,NE,OMAHA EPPLEY AIRFIELD,NaN,NaN,72550.0,US,United States


In [16]:
statesDF = pd.read_fwf('ghcnd-states.txt',header=None,delimiter=' ',names=['State','StateName'])
statesDF.head()

,State,StateName
0,AB,ALBERTA
1,AK,ALASKA
2,AL,ALABAMA
3,AR,ARKANSAS
4,AS,AMERICAN SAMOA


In [17]:
stations_DF2 = pd.merge(stations_DF2,statesDF,on='State',how='left')
stations_DF2[stations_DF2['StationName'].str.contains('EPPLEY')]

,Latitude,Longitude,Elevation,State,StationName,GSN_Flag,HCN_CRN_Flag,WMO_ID,CountryCode,CountryName,StateName
104868,41.3103,-95.8992,299.3,NE,OMAHA EPPLEY AIRFIELD,NaN,NaN,72550.0,US,United States,NEBRASKA


In [18]:
# Write file to local directory
stationsDF.to_csv('ghcnd-stations-cleansed.csv',sep='\t')